In [ ]:
from tensorflow import keras
import numpy as np

# Load the Data
(Xtr, ytr), (Xte, yte) = keras.datasets.mnist.load_data()

# Reshaping the (60000, 28, 28)  to (60000, 784)
Xtr = Xtr.reshape(Xtr.shape[0], -1)
Xte = Xte.reshape(Xte.shape[0], -1)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model_KNN = KNeighborsClassifier(n_neighbors=3)
model_KNN.fit(Xtr, ytr)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
from PIL import Image 

def resize(names):
    for val in names: 
        image = Image.open("../data/"+val+".png")
        resized_image = image.resize((28,28))
        resized_image.save("../data/"+val+"_28x28.png")

def load(names):
    images=list()
    for val in names: 
        image = Image.open("../data/"+val+".png")
        image_narray = np.array(image).reshape(1,-1)
        image_narray = 1-image_narray
        images.append( (image, image_narray) )
    return images

In [ ]:
import tkinter as tk
from tkinter import Canvas, Label
from PIL import Image, ImageDraw


class DrawingApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Number",)

        self.canvas_width = 280
        self.canvas_height = 200

        self.root.wm_attributes('-toolwindow', 'True')



        # set minimum window size value
        self.root.minsize(300, 260)
        
        # set maximum window size value
        self.root.maxsize(300, 260)

        self.canvas = Canvas(root, width=self.canvas_width, height=self.canvas_height, bg="white")
        self.canvas.pack(expand=tk.YES, fill=tk.BOTH)

        self.image = Image.new("L", (self.canvas_width, self.canvas_height), "white")
        self.draw = ImageDraw.Draw(self.image)
        

        self.old_x = None
        self.old_y = None
        self.pen_color = "black"
        self.pen_width = 15

        self.canvas.bind("<B1-Motion>", self.paint)
        self.canvas.bind("<ButtonRelease-1>", self.reset)

        save_button = tk.Button(
            root, 
            text=" Predict ", 
            font=("Lucida Grande", 10),
            relief="groove",
            bg="white",
            fg="black",
            activeforeground="white",
            activebackground="black",
            command=self.save_drawing
        )
        save_button.pack(side="left")

        clear_screen= tk.Button(
            root, 
            text="  Clear  ", 
            font=("Lucida Grande", 10),
            relief="groove",
            bg="white",
            fg="black",
            activeforeground="white",
            activebackground="black",
            command=self.clear_drawing
        )
        clear_screen.pack(side="left")
        

        self.L = Label(
            root, 
            font=("Lucida Grande", 16),
            height = 1, 
            width = 2,
        )
        self.L.pack(side="left")



    def paint(self, event):
        if self.old_x and self.old_y:
            self.canvas.create_line(self.old_x, self.old_y, event.x, event.y,
                                    width=self.pen_width, fill=self.pen_color,
                                    capstyle=tk.ROUND, smooth=tk.TRUE)
            self.draw.line([self.old_x, self.old_y, event.x, event.y],
                           fill=self.pen_color, width=self.pen_width)
        self.old_x = event.x
        self.old_y = event.y

    def reset(self, event):
        self.old_x, self.old_y = None, None

    def save_drawing(self):
        self.image.save("../data/predict.png")
        resize(["predict"])
        images = load(["predict_28x28"])
        y_hat = model_KNN.predict(images[0][1])
        self.L.config(text=f" {y_hat[0]}")
        

    def clear_drawing(self):
        
        self.canvas.delete("all")
        self.image = Image.new("L", (self.canvas_width, self.canvas_height), "white")
        self.draw = ImageDraw.Draw(self.image)
        self.L.config(text=f" ")



if __name__ == "__main__":

    root = tk.Tk()
    app = DrawingApp(root)
    root.mainloop()